In [7]:
# Define a workspace and the dataset

from azureml.core import Workspace

print("Accessing the workspace...")
ws = Workspace.from_config(path="./config")

print("Accessing the dataset...")
input_ds = ws.datasets.get("gfdhnbv")

Accessing the workspace...
Accessing the dataset...


In [8]:
# Create a compute cluster

cluster_name = "mlclustertemp"

from azureml.core.compute import AmlCompute

print("Accessing the compute cluster...")

if cluster_name not in ws.compute_targets:
    print("Creating the compute cluster with name: ", cluster_name)
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D11_V2", max_nodes=2)
    cluster = AmlCompute.create(ws, cluster_name, compute_config)
    cluster.wait_for_completion()
else:
    cluster = ws.compute_targets[cluster_name]
    print(cluster_name, ", compute cluster found. Using it...")

Accessing the compute cluster...
mlclustertemp , compute cluster found. Using it...


In [9]:
# Configure the AutoML run

from azureml.train.automl import AutoMLConfig

print("Creating the AutoML Configuration...")

automl_config = AutoMLConfig(task='classification',
                             compute_target=cluster,
                             training_data=input_ds,
                             validation_size=0.3,
                             label_column_name="Default Next Month",
                             primary_metric="norm_macro_recall",
                             iterations=10,
                             max_concurrent_iterations=2,
                             experiment_timeout_hours=0.25,
                             featurization="auto")

Creating the AutoML Configuration...


In [10]:
# Create and submit the experiment
from azureml.core.experiment import Experiment

new_exp = Experiment(ws, "azureml-sdk-exp-001")

print("Submitting the experiment...")

new_run = new_exp.submit(automl_config)

new_run.wait_for_completion(show_output=True)

Submitting the experiment...
Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
azureml-sdk-exp-001,AutoML_6dde8816-5d1b-407b-8cb3-c6bdd749f5f8,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


Experiment,Id,Type,Status,Details Page,Docs Page
azureml-sdk-exp-001,AutoML_6dde8816-5d1b-407b-8cb3-c6bdd749f5f8,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturizationCompleted. Beginning model selection.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to customize the handling of missing feature values that may be more appropriate based on the data type and business requirement.
              Learn mor

{'runId': 'AutoML_6dde8816-5d1b-407b-8cb3-c6bdd749f5f8',
 'target': 'mlclustertemp',
 'status': 'Completed',
 'startTimeUtc': '2024-04-05T17:45:02.129963Z',
 'endTimeUtc': '2024-04-05T17:54:33.147791Z',
 'services': {},
 'properties': {'num_iterations': '10',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'norm_macro_recall',
  'train_split': '0.3',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'mlclustertemp',
  'AMLSettingsJsonString': '{"path":null,"name":"azureml-sdk-exp-001","subscription_id":"49e690c2-0fc4-42a6-ba64-9ea0fa04ddda","resource_group":"AzureMLsdkgroup","workspace_name":"slavasdkworkspace","region":"westus","compute_target":"mlclustertemp","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":true,"enable_parallel_run":false,"num_procs":null,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_gen

In [15]:
# Retrieve the best model
best_child = new_run.get_best_child()
best_child

Experiment,Id,Type,Status,Details Page,Docs Page
azureml-sdk-exp-001,AutoML_6dde8816-5d1b-407b-8cb3-c6bdd749f5f8_8,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [16]:
# Get the metrics for all the runs

for run in new_run.get_children():
    print("")
    print("Run ID : ", run.id)
    print(run.get_metrics("accuracy"))
    print(run.get_metrics("norm_macro_recall"))


Run ID :  AutoML_6dde8816-5d1b-407b-8cb3-c6bdd749f5f8_9
{'accuracy': 0.5979166666666667}
{'norm_macro_recall': 0.13918878014327518}

Run ID :  AutoML_6dde8816-5d1b-407b-8cb3-c6bdd749f5f8_8
{'accuracy': 0.6208333333333333}
{'norm_macro_recall': 0.20240137221269294}

Run ID :  AutoML_6dde8816-5d1b-407b-8cb3-c6bdd749f5f8_7
{'accuracy': 0.7708333333333334}
{'norm_macro_recall': 0.04338613661588142}

Run ID :  AutoML_6dde8816-5d1b-407b-8cb3-c6bdd749f5f8_6
{'accuracy': 0.7791666666666667}
{'norm_macro_recall': 0.0}

Run ID :  AutoML_6dde8816-5d1b-407b-8cb3-c6bdd749f5f8_5
{'accuracy': 0.55625}
{'norm_macro_recall': 0.10599334073251931}

Run ID :  AutoML_6dde8816-5d1b-407b-8cb3-c6bdd749f5f8_4
{'accuracy': 0.6125}
{'norm_macro_recall': 0.17142568862879637}

Run ID :  AutoML_6dde8816-5d1b-407b-8cb3-c6bdd749f5f8_worker_1
{}
{}

Run ID :  AutoML_6dde8816-5d1b-407b-8cb3-c6bdd749f5f8_3
{'accuracy': 0.6083333333333333}
{'norm_macro_recall': 0.1795984259913228}

Run ID :  AutoML_6dde8816-5d1b-407b-8c